In [1]:
import animationwidget
dir(animationwidget)

['AnimationWidget',
 '__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 '__path__',
 '__version__',
 '_jupyter_nbextension_paths',
 '_version',
 'example',
 'traitlets',
 'version_info',
 'widgets']

In [3]:
w = animationwidget.AnimationWidget()

In [4]:
w

In [8]:
w.run = True

In [9]:
w.run = False